# Mini RAG Assistant - Colab Notebook
This notebook implements a simplified Retrieval-Augmented Generation (RAG) system using FAISS, Sentence Transformers, and OpenAI API.

In [ ]:
# Step 1: Install dependencies
!pip install sentence-transformers faiss-cpu openai pandas

: 

In [ ]:
# Step 2: Import libraries
import openai
import faiss
import os
import pandas as pd
from datetime import datetime
from sentence_transformers import SentenceTransformer
from google.colab import files

In [ ]:
# Step 3: Upload .txt/.md files
uploaded = files.upload()
documents = []
for filename in uploaded:
    with open(filename, 'r', encoding='utf-8') as f:
        documents.append(f.read())

In [ ]:
# Step 4: Chunk the text
def chunk_text(text, chunk_size=300):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunks = []
for doc in documents:
    chunks.extend(chunk_text(doc))
print(f"Total Chunks: {len(chunks)}")

In [ ]:
# Step 5: Generate embeddings and store in FAISS index
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

In [ ]:
# Step 6: Ask a question and retrieve top 2 relevant chunks
question = input("Enter your question: ")
q_embedding = model.encode([question])
_, I = index.search(q_embedding, k=2)

retrieved_chunks = [chunks[i] for i in I[0]]
context = "\n".join(retrieved_chunks)

In [ ]:
# Step 7: Generate answer using OpenAI GPT
openai.api_key = "your-openai-api-key"  # Replace this with your actual API key

prompt = f"""Answer the question based on the following context:
{context}

Question: {question}
Answer:"""

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
)

answer = response['choices'][0]['message']['content']
print("\nAnswer:\n", answer)

In [ ]:
# Step 8: Log the interaction
log_entry = {
    "timestamp": datetime.now(),
    "question": question,
    "answer": answer
}

log_df = pd.DataFrame([log_entry])
log_df.to_csv("qa_log.csv", mode='a', header=not os.path.exists("qa_log.csv"), index=False)
files.download("qa_log.csv")